# Victorian Crime Statistics - Criminal Incidents by Principal Offence (2010-2019)

This notebook analyzes the VIC CSA Crime Statistics dataset covering Criminal Incidents by Principal Offence for Local Government Areas (LGAs) from 2010-2019.

## Data Source

- **Dataset:** VIC CSA - Crime Statistics - Criminal Incidents by Principal Offence (LGA) 2010-2019
- **Publisher:** Crime Statistics Agency Victoria
- **Time Period:** Years ending March 2010-2019
- **Geographic Boundaries:** ASGS 2011 LGA boundaries

### Download Sources

1. [AURIN Data Catalogue](https://data.aurin.org.au/dataset/vic-govt-csa-csa-crime-stats-criminal-incidents-princ-offence-lga-2010-2019-lga2011)
2. [data.gov.au](https://data.gov.au/dataset/ds-aurin-55c99905-75fe-49b8-a663-85b6f24b827d)
3. [Crime Statistics Agency Victoria](https://www.crimestatistics.vic.gov.au/download-data-11)

## Analysis Objectives

1. Analyze crime trends by principal offence division (2010-2019)
2. Compare crime rates across LGAs
3. Identify temporal patterns in offence types
4. Merge with 2012-2021 data for extended time series analysis

## Setup and Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')

# Set up paths
PROJECT_ROOT = Path('..').resolve()
DATA_DIR = PROJECT_ROOT / 'Data'
OUTPUT_DIR = PROJECT_ROOT / 'Output_data'

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")

In [ ]:
# Import the data processor module
import sys
sys.path.insert(0, str(PROJECT_ROOT / 'Scripts'))

from data_ingestion import CrimeDataProcessor

# Initialize the processor
processor = CrimeDataProcessor()

# Check available data files
available_files = processor.get_available_files()
print("Available data files:")
for name, exists in available_files.items():
    status = "OK" if exists else "NOT FOUND"
    print(f"  [{status}] {name}")

## 1. Load 2012-2021 Criminal Incidents Data (Reference)

First, let's load the existing 2012-2021 data to understand the structure and compare with the 2010-2019 dataset.

In [ ]:
# Load the 2012-2021 criminal incidents data
df_2012_2021 = processor.load_criminal_incidents_2012_2021(sheet_name="Table 01")

print(f"Loaded {len(df_2012_2021):,} rows")
print(f"\nColumns: {list(df_2012_2021.columns)}")
print(f"\nYears: {sorted(df_2012_2021['year'].dropna().unique())}")
print(f"\nUnique LGAs: {df_2012_2021['lga_name'].nunique()}")

df_2012_2021.head()

## 2. Load 2010-2019 Criminal Incidents by Principal Offence

Now let's load the 2010-2019 dataset. If the file is not available, you'll see instructions on how to download it.

In [ ]:
# Try to load the 2010-2019 data
try:
    df_2010_2019 = processor.load_criminal_incidents_2010_2019()
    data_2010_available = True
    
    print(f"Loaded {len(df_2010_2019):,} rows")
    print(f"\nColumns: {list(df_2010_2019.columns)}")
    print(f"\nYears: {sorted(df_2010_2019['year'].dropna().unique())}")
    
except FileNotFoundError as e:
    data_2010_available = False
    print("2010-2019 Data Not Available")
    print("=" * 50)
    print("\nPlease download the data from one of these sources:")
    print("\n1. AURIN Data Catalogue (requires institutional login):")
    print("   https://data.aurin.org.au/dataset/vic-govt-csa-csa-crime-stats-criminal-incidents-princ-offence-lga-2010-2019-lga2011")
    print("\n2. Crime Statistics Agency Victoria:")
    print("   https://www.crimestatistics.vic.gov.au/download-data-11")
    print("\nSave the file as:")
    print(f"   {DATA_DIR}/LGA_Criminal_Incidents_Principal_Offence_2010_2019.xlsx")
    print("\nAlternatively, run from command line:")
    print("   python Scripts/data_ingestion/download_2010_2019_data.py --download")

## 3. Analyze Crime Trends (2012-2021)

While waiting for the 2010-2019 data, let's analyze the existing 2012-2021 trends.

In [ ]:
# Aggregate incidents by year
yearly_totals = df_2012_2021.groupby('year').agg({
    'incidents_recorded': 'sum'
}).reset_index()

# Create the trend plot
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(yearly_totals['year'], yearly_totals['incidents_recorded'], 
        marker='o', linewidth=2, markersize=8, color='#1f77b4')

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Total Criminal Incidents', fontsize=12)
ax.set_title('Victorian Criminal Incidents Trend (2012-2021)', fontsize=14, fontweight='bold')

# Format y-axis with commas
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

# Add data labels
for x, y in zip(yearly_totals['year'], yearly_totals['incidents_recorded']):
    ax.annotate(f'{y:,.0f}', (x, y), textcoords="offset points", 
                xytext=(0, 10), ha='center', fontsize=9)

ax.set_xticks(yearly_totals['year'])
plt.tight_layout()
plt.show()

print("\nYear-over-Year Change:")
yearly_totals['yoy_change'] = yearly_totals['incidents_recorded'].pct_change() * 100
print(yearly_totals.to_string(index=False))

## 4. Crime Rates by Police Region

In [ ]:
# Aggregate by Police Region and Year
region_trends = df_2012_2021.groupby(['year', 'police_region']).agg({
    'incidents_recorded': 'sum'
}).reset_index()

# Create line plot for each region
fig, ax = plt.subplots(figsize=(14, 7))

regions = region_trends['police_region'].unique()
colors = plt.cm.Set1(np.linspace(0, 1, len(regions)))

for region, color in zip(regions, colors):
    region_data = region_trends[region_trends['police_region'] == region]
    ax.plot(region_data['year'], region_data['incidents_recorded'], 
            marker='o', linewidth=2, label=region, color=color)

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Total Criminal Incidents', fontsize=12)
ax.set_title('Criminal Incidents by Police Region (2012-2021)', fontsize=14, fontweight='bold')

ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.legend(title='Police Region', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.set_xticks(region_trends['year'].unique())

plt.tight_layout()
plt.show()

## 5. Top LGAs by Crime Rate

In [ ]:
# Get latest year's data
latest_year = df_2012_2021['year'].max()
df_latest = df_2012_2021[df_2012_2021['year'] == latest_year]

# Filter out aggregated rows
df_latest_lga = df_latest[~df_latest['lga_name'].isin(['Total', 'Justice Institutions and Immigration Facilities', 'Unincorporated Vic'])]

# Sort by crime rate
top_15 = df_latest_lga.nlargest(15, 'rate_per_100000')

# Create horizontal bar chart
fig, ax = plt.subplots(figsize=(12, 8))

bars = ax.barh(range(len(top_15)), top_15['rate_per_100000'], color='#e74c3c')
ax.set_yticks(range(len(top_15)))
ax.set_yticklabels(top_15['lga_name'])
ax.invert_yaxis()

ax.set_xlabel('Rate per 100,000 Population', fontsize=12)
ax.set_title(f'Top 15 LGAs by Crime Rate ({latest_year})', fontsize=14, fontweight='bold')

# Add value labels
for i, (v, region) in enumerate(zip(top_15['rate_per_100000'], top_15['police_region'])):
    ax.text(v + 50, i, f'{v:,.0f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\nTop 15 LGAs by Crime Rate ({latest_year}):")
print(top_15[['lga_name', 'police_region', 'incidents_recorded', 'rate_per_100000']].to_string(index=False))

## 6. Analysis by Offence Division (Table 02)

Load and analyze crime by offence type.

In [ ]:
# Load data by offence type
df_offence = processor.load_criminal_incidents_2012_2021(sheet_name="Table 02")

print(f"Loaded {len(df_offence):,} rows with offence breakdown")
print(f"\nOffence Divisions:")
for div in sorted(df_offence['offence_division'].unique()):
    count = df_offence[df_offence['offence_division'] == div]['incidents_recorded'].sum()
    print(f"  {div}: {count:,.0f} incidents")

In [ ]:
# Aggregate by offence division and year
offence_trends = df_offence.groupby(['year', 'offence_division']).agg({
    'incidents_recorded': 'sum'
}).reset_index()

# Create stacked area chart
fig, ax = plt.subplots(figsize=(14, 8))

pivot_df = offence_trends.pivot(index='year', columns='offence_division', values='incidents_recorded')
pivot_df.plot.area(ax=ax, alpha=0.7, colormap='Set2')

ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Criminal Incidents', fontsize=12)
ax.set_title('Criminal Incidents by Offence Division (2012-2021)', fontsize=14, fontweight='bold')

ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.legend(title='Offence Division', bbox_to_anchor=(1.02, 1), loc='upper left', fontsize=9)

plt.tight_layout()
plt.show()

## 7. Merge Datasets for Extended Time Series (2010-2021)

If the 2010-2019 data is available, merge it with the 2012-2021 data for an extended analysis.

In [ ]:
# Try to merge datasets
try:
    df_merged = processor.merge_time_series(prefer_september=True)
    
    print(f"Successfully merged datasets!")
    print(f"Total rows: {len(df_merged):,}")
    print(f"\nYear range: {df_merged['year'].min()} - {df_merged['year'].max()}")
    print(f"\nData sources:")
    print(df_merged['data_source'].value_counts())
    
except Exception as e:
    print(f"Could not merge datasets: {e}")
    print("\nUsing 2012-2021 data only for analysis.")
    df_merged = df_2012_2021.copy()

## 8. Export Processed Data

In [ ]:
# Export LGA summary to CSV
lga_summary = processor.get_lga_summary(df_2012_2021)

output_path = OUTPUT_DIR / 'criminal_incidents_by_lga_2012_2021.csv'
lga_summary.to_csv(output_path, index=False)
print(f"Exported LGA summary to: {output_path}")

# Display sample
print(f"\nSample of exported data ({len(lga_summary):,} rows):")
lga_summary.head(10)

## 9. Summary Statistics

In [ ]:
# Generate summary report
report = processor.generate_report()

print("\nData Processing Report")
print("=" * 50)
print(f"Generated at: {report['generated_at']}")

print("\nData Files:")
for name, info in report['data_files'].items():
    if info['exists']:
        print(f"  [{name}] {info['size_mb']:.2f} MB")
    else:
        print(f"  [{name}] NOT FOUND")

print("\nLoaded Datasets:")
for name, info in report['loaded_data'].items():
    print(f"  [{name}]")
    print(f"    Rows: {info['rows']:,}")
    if info['years']:
        print(f"    Years: {min(info['years'])} - {max(info['years'])}")

## Next Steps

1. **Download 2010-2019 Data:** If not already available, download from the Crime Statistics Agency Victoria
2. **Extended Analysis:** Once merged, analyze the full 2010-2021 time series
3. **Geographic Visualization:** Use the LGA boundaries GeoJSON for spatial analysis
4. **Offence Type Deep Dive:** Analyze specific offence types across time

### Data Sources

- [Crime Statistics Agency Victoria](https://www.crimestatistics.vic.gov.au/)
- [AURIN Data Catalogue](https://data.aurin.org.au/)
- [data.gov.au](https://data.gov.au/)